# Store Item Sales Prediction 

### Import packages

In [ ]:
import os 

import math
import numpy as np
import pandas as pd
import sklearn

import matplotlib.pyplot as plt
import seaborn as sns

# from pandas_profiling import ProfileReport

from sklearn.model_selection import train_test_split

from sklearn.feature_selection import f_regression, mutual_info_regression


from sklearn.feature_extraction import DictVectorizer



from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_text
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import RandomizedSearchCV

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [ ]:
pd.set_option('display.precision', 3)

In [ ]:
path = os.getcwd()
os.listdir(path)

### Convert column names to lower case 
For the sake of convenience

In [ ]:
def std_col_names(df):
    df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
    return df

### Load data

In [ ]:
df = std_col_names(pd.read_csv('train.csv'))


In [ ]:
x_test = std_col_names(pd.read_csv('test.csv'))


### Observe data

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

- Minimum sales value is -1230
- Check for number of items with negative sales values


In [ ]:
a= (df['sales']<0).value_counts(normalize=True)
print(f'Ratio of negative sales values: {a.loc[1]:.3f}')

Ratio of negative sales values is negligible

In [ ]:
df.hist(bins= 50, figsize= (20,15))

Sales is right skewed distribution

In [ ]:
# profile = ProfileReport(df)
# profile.to_file("sales_analysis.html")

### Split data

In [ ]:
# Split data into train, validation
train, val = train_test_split(df, random_state=42, test_size=0.15)

In [ ]:
# Reset index to be sequenced
train = train.reset_index(drop= True)
val=val.reset_index(drop= True)

#### Drop the target column from train and validation feature sets


In [ ]:
x_train = train.drop('sales', axis= 1)
y_train = train['sales'].copy()

x_val = val.drop('sales', axis= 1)
y_val = val['sales'].copy()

In [ ]:
len(df), len(x_train), len(x_val)

### Extract numerical | categorical columns

In [ ]:
num_cols = x_train.select_dtypes(include=['int64', 'float64']).columns.to_list()
cat_cols = x_train.select_dtypes(include=['object']).columns.to_list()

### Check the sales of items as a function of outlet attributes (Size | Location type)

In [ ]:
outlet_df = (train.pivot_table( values = ['sales'], columns= ['outlet_location_type',\
                                                             'outlet_size'],\
                              aggfunc= 'sum')/math.pow(10, 6)).T.reset_index()
outlet_df

In [ ]:
outlet_df.pivot("outlet_location_type", "outlet_size", "sales").plot(kind='bar', ylabel= 'Sales', title= 'Sales vs Location type | size')

In [ ]:
sales_by_location = outlet_df.groupby(by='outlet_location_type').agg('sum')
sales_by_location.plot(kind= 'bar', title= 'Sales by location type', ylabel='Sales(M)')

In [ ]:
location_count = train['outlet_location_type'].value_counts()
location_count.plot(kind= 'bar', title= 'Store count by location type')


In [ ]:
sales_by_size = outlet_df.groupby(by='outlet_size').agg('sum')
sales_by_size.plot(kind='bar', ylabel= 'Sales(M)')

In [ ]:
location_count = train['outlet_location_type'].value_counts()
location_count.plot(kind= 'bar', title= 'Store count by location type')
location_count

In [ ]:
size_count = train['outlet_size'].value_counts()
size_count.plot(kind= 'bar', title= 'Store count by size')

### Insights on Sales vs Outlet
From the above pivot graphs, we get the following insights:

- The small stores in Tier 1 and Tier 2 locations account for exceptionally higher sales
- Small stores in aggregation account for highest sales
- Aggregated sales are in order (Tier 1->3->2)

## Helper Functions

In [ ]:
def eval_rmse(model, train, val, train_actual= y_train, val_actual= y_val ):
    train_pred = model.predict(train)
    train_rmse = mean_squared_error(train_actual, train_pred, squared= False)
#     yield(f'{model} Train RMSE: {train_rmse}')

    val_pred = model.predict(val)
    val_rmse = mean_squared_error(val_actual,val_pred, squared= False)
#     yield(f'{model} Validation RMSE: {val_rmse}')
    
    return {f'{model}':(train_rmse, val_rmse)}

In [ ]:
def retrieve_item_cats(df, col):
    item_cats={}
    for x in range(4):
        if x<3:
            item_cats[f'item_cat_{x}'] = [df[col][idx][x] for idx in range(len(df))]
        else:
            item_cats[f'item_cat_{x}'] = [df[col][idx][x:] for idx in range(len(df))]
    return item_cats
            
            

In [ ]:
def process_data(df, cols):
    df_ids =  pd.DataFrame(retrieve_item_cats(df, 'item_id'))
    df_id_split = pd.concat([df, df_ids], axis= 1)
    df_id_split['out_size'] = df['outlet_size'].map(outlet_size_map)
    df_id_split['out_type'] = df['outlet_location_type'].map(outlet_location_map)
    df_id_split['outlet_age']= 2021-df_id_split['outlet_year']
    return df_id_split[cols]

In [ ]:
def export(series, name):
    sol = pd.DataFrame(series, columns= ['Sales'])


    sub = sol.to_csv(f'./subs/{name}.csv', index= False)

In [ ]:
def ord_encode(df):
    outlet_size_map = {'Small': 1, 'Medium': 2, 'High': 3}
    outlet_location_map = {'Tier 1': 3,'Tier 2': 2, 'Tier 3': 1}
    df['out_size'] = df['outlet_size'].map(outlet_size_map)
    df['out_type'] = df['outlet_location_type'].map(outlet_location_map)
    return df

In [ ]:
def scale_num_cols(df, scale_cols):
    for col in scale_cols:
        df[col+'_scale'] = (df[col] - min(df[col])) / (max(df[col]) - min(df[col]))
    return df

## Approach 1: Innocent (without `item_id`)

In [ ]:
len(num_cols), len(cat_cols)

### Initialize DictVectorizer for OneHotEncoding

In [ ]:

dv = DictVectorizer(sparse=False)

train_dict = x_train[x_train.columns.to_list()[1:]].to_dict(orient= 'records')
train_dv = dv.fit_transform(train_dict)

val_dict = x_val[x_train.columns.to_list()[1:]].to_dict(orient= 'records')
val_dv = dv.transform(val_dict)

In [ ]:
(f'Number of columns after OneHotEncoding: {len(dv.feature_names_)}')

### LinearRegression 

In [ ]:
lr= LinearRegression()

In [ ]:
lr.fit(train_dv, y_train)

In [ ]:
lr_train_preds = lr.predict(train_dv)


In [ ]:
lr_rmse = eval_rmse(lr, train_dv, val_dv)
lr_rmse

In [ ]:
# # Predict on test set
# test_dict = x_test[x_test.columns.to_list()[1:]].to_dict(orient= 'records')
# test_dv = dv.transform(test_dict)

# dt_test_preds = dt.predict(test_dv)

# sol = pd.DataFrame(dt_test_preds, columns= ['Sales'])


# sub = sol.to_csv('./subs/dt_noItemID.csv', index= False)

### DecisionTree

In [ ]:
dt = DecisionTreeRegressor()

In [ ]:
dt.fit(train_dv, y_train)

In [ ]:
dt_train_preds = dt.predict(train_dv)


In [ ]:
dt_rmse = eval_rmse(dt, train_dv, val_dv)
dt_rmse

In [ ]:
# test_dict = x_test[x_test.columns.to_list()[1:]].to_dict(orient= 'records')
# test_dv = dv.transform(test_dict)

# lr_test_preds = lr.predict(test_dv)

# sol = pd.DataFrame(lr_test_preds, columns= ['Sales'])


# sub = sol.to_csv('./subs/lr_noItemID.csv', index= False)

## Approach 2:  Better pre-processing
Let's look at the dataset again

In [ ]:
x_train.head()

### Observe sales based on `item_type`

In [ ]:
item_sales_df = (train[['item_type', 'sales']].groupby(by= 'item_type').agg('sum'))/math.pow(10,6)
item_sales_df = item_sales_df.sort_values(by='sales', ascending= False)
item_sales_df['sales_ratio'] = (item_sales_df['sales']/ item_sales_df['sales'].sum())*100
item_sales_df

In [ ]:
unique_item_types = train['item_type'].nunique()
print(f'Unique_item_types: {unique_item_types }')
unique_item_ids = train['item_id'].nunique()
print(f'Unique_item_ids: {unique_item_ids }')


### Split the `item_id` feature into more categorical parts

In [ ]:
x_train_ids = pd.DataFrame(retrieve_item_cats(x_train, 'item_id'))


In [ ]:
x_train_ids.head()

In [ ]:
x_train_id_split = pd.concat([x_train, x_train_ids], axis= 1)

In [ ]:
x_train_ids.nunique()

- We have expanded the `item_id` column now. As observed above, the unique number of values in the 4 columns should ideally be lesser than the number of unique columns when they are combined (`item_id` as a whole).   
<br>

- The number of unique values for the split columns is `(3+3+26+60)= 92`  
<br>

- Let's check for item_id as a single column.

In [ ]:
x_train['item_id'].nunique()

894 unique values for `item_id`! This is bound to create too many features and mostly not be able to capture patterns

In [ ]:
item_id_counts = pd.DataFrame(train['item_id'].value_counts()).sort_index()
item_id_counts

In [ ]:
pd.DataFrame(train.groupby(by='item_id').agg('sum')['sales'] / item_id_counts['item_id']).sort_values(by=0,ascending=False)

In [ ]:
x_train_id_split.head()

## Ordinal Encode `outlet_size` and `outlet_location_type`

In [ ]:
outlet_size_map = {'Small': 1, 'Medium': 2, 'High': 3}
outlet_location_map = {'Tier 1': 3,'Tier 2': 2, 'Tier 3': 1}

In [ ]:
x_train_id_split['out_size'] = x_train['outlet_size'].map(outlet_size_map)
x_train_id_split['out_type'] = x_train['outlet_location_type'].map(outlet_location_map)

### Convert `oulet_year` to `outlet_age`

In [ ]:
x_train_id_split['outlet_age']= 2021-x_train_id_split['outlet_year']
x_train_id_split.head()

In [ ]:
x_train_id_split['item_cat_3'] = x_train_id_split['item_cat_3'].astype('str')

### Transformations done:
- Split item_id -> `retrieve_item_cats`
- Ordinally encode -> `pd.map`
- Outlet age -> `2021 - x['outlet_year']`
- One hot encode/DictVectorize columns -> use `cols`

In [ ]:
x_train_id_split.columns

### Initial set of columns

In [ ]:

col = ['item_cat_0',
       'item_cat_1', 'item_cat_2', 'item_cat_3',  'item_w', 'item_type', 'item_mrp', 'outlet_id',
       'out_size', 'out_type','outlet_age']

In [ ]:
# train_dict_ext = x_train_id_split.to_dict(orient= 'records')
dv_ext = DictVectorizer()

train_dict_ext = x_train_id_split[col].to_dict(orient= 'records')
train_dv_ext = dv_ext.fit_transform(train_dict_ext)

# val_dict = x_val[x_train.columns.to_list()[1:]].to_dict(orient= 'records')
# val_dv = dv.transform(val_dict)

In [ ]:
(f'Number of columns after all processing: {len(dv_ext.get_feature_names())}')

#### Preprocessing validation data

In [ ]:
x_val.head()

### Perform transformations

In [ ]:
x_val_ids =  pd.DataFrame(retrieve_item_cats(x_val, 'item_id'))
x_val_id_split = pd.concat([x_val, x_val_ids], axis= 1)
x_val_id_split['out_size'] = x_val['outlet_size'].map(outlet_size_map)
x_val_id_split['out_type'] = x_val['outlet_location_type'].map(outlet_location_map)
x_val_id_split['outlet_age']= 2021-x_val_id_split['outlet_year']
# x_val_id_split['item_cat_3'] = x_val_id_split['item_cat_3'].astype('str')

In [ ]:
x_val_id_split[col].head()

In [ ]:
x_val_id_split[col].info()

In [ ]:
val_dict_ext = x_val_id_split[col].to_dict(orient= 'records')
val_dv_ext = dv_ext.transform(val_dict_ext)

####  Linear Regression

In [ ]:
lr_ext = LinearRegression()

In [ ]:
lr_ext.fit(train_dv_ext, y_train)

In [ ]:
lr_train_preds_ext = lr_ext.predict(train_dv_ext)

In [ ]:
lr_val_preds_ext = lr_ext.predict(val_dv_ext)

In [ ]:
print(mean_squared_error(y_train, lr_train_preds_ext, squared= False))
print(mean_squared_error(y_train, lr_train_preds, squared= False))

# eval_rmse(lr_ext, lr_train_preds_ext.reshape(-1,1), lr_val_preds_ext.reshape(-1,1))

In [ ]:
lr_val_preds = lr.predict(val_dv)
print(mean_squared_error(y_val, lr_val_preds_ext, squared=False))
print(mean_squared_error(y_val, lr_val_preds, squared=False))

### Feature importance for LinearRegression

In [ ]:
pd.DataFrame(list(zip(dv_ext.get_feature_names(), abs(lr_ext.coef_) ))).sort_values(by=1, ascending=False )[-50:]

## Testing Models

### Data

In [ ]:
x_test.head()


In [ ]:
x_test_ids =  pd.DataFrame(retrieve_item_cats(x_test, 'item_id'))
x_test_id_split = pd.concat([x_test, x_test_ids], axis= 1)
x_test_id_split['out_size'] = x_test['outlet_size'].map(outlet_size_map)
x_test_id_split['out_type'] = x_test['outlet_location_type'].map(outlet_location_map)
x_test_id_split['outlet_age']= 2021-x_test_id_split['outlet_year']
# x_val_id_split['item_cat_3'] = x_val_id_split['item_cat_3'].astype('str')

In [ ]:
test_dict_ext = x_test_id_split[col].to_dict(orient= 'records')
test_dv_ext = dv_ext.transform(test_dict_ext)

In [ ]:
lr_test_preds_ext = lr_ext.predict(test_dv_ext)

#### Decision Trees

In [ ]:
dt_ext = DecisionTreeRegressor(max_depth= 5, max_features=10, ccp_alpha=0.25 )

dt_ext.fit(train_dv_ext, y_train)

In [ ]:
# pd.DataFrame(list(zip(dv_ext.get_feature_names(), abs(dt_ext.feature_importances_)))).sort_values(by= 1, ascending=False)

In [ ]:
dt_train_preds_ext = dt_ext.predict(train_dv_ext)

dt_val_preds_ext = dt_ext.predict(val_dv_ext)

In [ ]:
print(mean_squared_error(y_train, dt_train_preds_ext, squared= False))
print(mean_squared_error(y_val, dt_val_preds_ext, squared= False))

In [ ]:
dt_test_preds_ext = dt_ext.predict(test_dv_ext)

#### Random Forests

In [ ]:
rf = RandomForestRegressor(n_estimators= 100, max_depth= 6)

rf.fit(train_dv_ext, y_train)

In [ ]:
rf_train_preds = rf.predict(train_dv_ext)

rf_val_preds= rf.predict(val_dv_ext)

In [ ]:
print(mean_squared_error(y_train, rf_train_preds, squared= False))
print(mean_squared_error(y_val, rf_val_preds, squared= False))

In [ ]:
rf_test_preds = rf.predict(test_dv_ext)

## Fit models to entire train dataset

In [ ]:
col = ['item_cat_0',
       'item_cat_1', 'item_cat_2', 'item_cat_3',  'item_w', 'item_type', 'item_mrp', 'outlet_id',
       'out_size', 'out_type','outlet_age']
scaled_cols = ['item_cat_0',
       'item_cat_1', 'item_cat_2', 'item_cat_3',  'item_w_scale', 'item_type', 'item_mrp_scale', 'outlet_id',
       'out_size', 'out_type','outlet_age_scale']
scale_cols = ['item_w', 'item_mrp', 'outlet_age']

In [ ]:
full_train = process_data(train, col)
full_train.head()

### Mutual Information - Regression

In [ ]:
mi_cols = [  'item_w',  'item_mrp', 
       'out_size', 'out_type','outlet_age']

X = full_train[mi_cols]
y = train['sales']

f_test, _ = f_regression(X, y)
f_test /= np.max(f_test)

mi = mutual_info_regression(X, y)
mi /= np.max(mi)

plt.figure(figsize=(15, 5))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.scatter(X.iloc[:, i], y, edgecolor="black", s=20)
    plt.xlabel("$x_{}$".format(i + 1), fontsize=14)
    if i == 0:
        plt.ylabel("$y$", fontsize=14)
    plt.title("F-test={:.2f}, MI={:.2f}".format(f_test[i], mi[i]), fontsize=16)
plt.show()



In [ ]:
list(zip(mi_cols, mi))

### Scale numerical columns using Min-Max Normalization

In [ ]:
for col in scale_cols:
    full_train[col+'_scale'] = (full_train[col] - min(full_train[col])) / (max(full_train[col]) - min(full_train[col]))
    x_test_id_split[col+'_scale'] = (x_test_id_split[col] - min(x_test_id_split[col])) / (max(x_test_id_split[col]) - min(x_test_id_split[col]))


In [ ]:
x_test_id_split[scaled_cols].head()

### Create DictVectorizer instance for entire train set

In [ ]:
full_dv = DictVectorizer()
full_train_dict = full_train[scaled_cols].to_dict(orient= 'records')
full_train_dv = full_dv.fit_transform(full_train_dict)

In [ ]:
full_test_dict = x_test_id_split[scaled_cols].to_dict(orient= 'records')
full_test_dv = full_dv.transform(full_test_dict)

In [ ]:
lr_full = LinearRegression()
lr_full.fit(full_train_dv, train['sales'])


lr_full_pred = lr_full.predict(full_train_dv)
mean_squared_error(train['sales'], lr_full_pred, squared=False)

In [ ]:
dt_full = DecisionTreeRegressor(max_depth= 5, max_features=10, ccp_alpha=0.01 )
dt_full.fit(full_train_dv, train['sales'])
dt_full_pred = dt_full.predict(full_train_dv)
mean_squared_error(train['sales'], dt_full_pred, squared=False)

### Display Decision Tree split as text

In [ ]:
print(export_text(dt_full, feature_names=full_dv.get_feature_names()))

In [ ]:
dt_full_test_pred = dt_full.predict(test_dv_ext)
# export(dt_full_test_pred, 'dt_full')

In [ ]:
full_train[col].head()

In [ ]:
train['sales']

In [ ]:
rf_full = RandomForestRegressor(max_depth= 5, n_estimators= 50)
rf_full.fit(full_train_dv, train['sales'])
rf_full_pred = rf_full.predict(full_train_dv)
mean_squared_error(train['sales'], rf_full_pred, squared=False)

### Parameter Tuning

Tuning takes time, hence below code block is commented. The best model is used to train data

In [ ]:

# rf_tune = RandomForestRegressor()
# distrib = {
#     'n_estimators': [10, 50, 100, 150, 200],
#     'max_depth': [5, 10, 15, 20],
#     'max_features':['sqrt', 'log2', 5, 10, 15],
#     'ccp_alpha' : [0.01, 0.05, 0.1, .05],
#     'max_samples': [0.15, 0.25, 0.33, 0.5]
# }

# tuner = RandomizedSearchCV(rf_tune, distrib, random_state = 42)

# tuner.fit(full_train_dv, train['sales'])

In [ ]:
# tuner.score

In [ ]:
rf_tuned = RandomForestRegressor(ccp_alpha=0.05, max_depth=15, max_features=5,
                      max_samples=0.5, n_estimators=200)

rf_tuned.fit(full_train_dv, train['sales'])
rf_full_pred = rf_tuned.predict(full_train_dv)


In [ ]:
rf_rmse= mean_squared_error( train['sales'], rf_full_pred, squared= False )
print(f'Tuned model RMSE: {round(rf_rmse, 3)}')

### Exporting to script
- We shall use the above model for making predictions. The training operations are exported to `train.py` script
<br>

- Further detailing is done in the [Wiki](https://github.com/ada-nai/ag-mlcamp/wiki/Store-Sales-Prediction#model-training)


### Making predictions on test set

In [ ]:
# rf_tuned_test_preds = rf_tuned.predict(full_test_dv)
# export(rf_tuned_test_preds, 'rf_tuned')

### Dump Objects

In [ ]:
# import pickle

# with open('model.bin', 'wb') as outfile:
#     pickle.dump(rf_tuned, outfile)
    
# with open('dv.bin', 'wb') as outfile:
#     pickle.dump(full_dv, outfile)
    
# with open('test_samples.bin', 'wb') as outfile:
#     pickle.dump(test_samples, outfile)

## Rough Work 

Below are some code snippets for quick prototyping of functions.  Code further on can be ignored

In [ ]:
test_samples= eval(df.sample(15, random_state= 42).to_json(orient= 'records'))
test_samples

In [ ]:
pd.DataFrame(test_samples).head()

In [ ]:
def retrieve_item_cats(df, col):
    """
    Split the item_id feature into categories
    """
    item_cats={}
    for x in range(4):
#         print(x)
        if x<3:
            item_cats[f'item_cat_{x}'] = [df[col][loc][x] for loc in range(len(df))]
        else:
            item_cats[f'item_cat_{x}'] = [df[col][loc][x:] for loc in range(len(df))]
    return item_cats

def scale_num_cols(df, scale_cols):
    for col in scale_cols:
        df[col+'_scale'] = (df[col] - min(df[col])) / (max(df[col]) - min(df[col]))
    return df

def process_data(df, cols):
    """

    """
    df = pd.DataFrame.from_dict(df).reset_index(drop= True)
    df_ids =  pd.DataFrame(retrieve_item_cats(df, 'item_id')).reset_index(drop= True)
    df_id_split = pd.concat([df, df_ids], axis= 1)
#     print([len(df), len(df_ids), len(df_id_split)])
    df_id_split['out_size'] = df['outlet_size'].map(outlet_size_map)
    df_id_split['out_type'] = df['outlet_location_type'].map(outlet_location_map)
    df_id_split['outlet_age']= 2021-df_id_split['outlet_year']
    df_id_split = scale_num_cols(df_id_split, scale_cols)

#     scale_num_cols(df_id_split, scale_cols)
    return df_id_split[scaled_cols]

In [ ]:
scale_cols = ['item_w', 'item_mrp', 'outlet_age']
scaled_cols = ['item_cat_0',
       'item_cat_1', 'item_cat_2', 'item_cat_3',  'item_w_scale', 'item_type', 'item_mrp_scale', 'outlet_id',
       'out_size', 'out_type','outlet_age_scale']
ip = process_data(test_samples, scaled_cols)

In [ ]:
# pd.DataFrame(test)
pd.DataFrame(test.values())